# Inference notebook

### Version 43

```text
2 fold + 2 fold mean

Local CV:   0.70639
Public LB:  0.72531
Private LB: 0.72432

y_pred = w1 * y_vit4 + w2 * y_unet1024 > 0.5

w1: 0.6465366913319027
w2: 0.4531613974630522
```

In [ ]:
! pip install -q --no-index --find-links=file:///kaggle/input/contrails-weights/pip \
    segmentation_models_pytorch
! cp -r /kaggle/input/contrails-src ./src

In [ ]:
import numpy as np
import sys
import yaml
sys.path.append('/kaggle/working/src')
import vit4
import unet1024
import util
from submit import write_submission

## Configuration

In [ ]:
cfg = yaml.safe_load("""
input:
  weight: /kaggle/input/contrails-weights

vit4:
  batch_size: 2
  models:
    - name: vit4_1024
      encoder: maxvit_tiny_tf_512.in1k
      decoder_channels: [256, 128, 64, 32, 32]
      w: 0.6465366913319027
      resize: 512
      tta: d4prob
      folds: [5, 7]

unet1024:
  batch_size: 2
  models:
    - name: unet1024
      encoder: maxvit_tiny_tf_512.in1k
      decoder_channels: [256, 128, 64, 32, 16]
      w: 0.4531613974630522
      resize: 1024
      tta: d4prob
      folds: [3, 4]
""")

## Run

In [ ]:
data_type = 'test'
#data_type = 'validation'
debug=False

# Ensemble
preds = vit4.run(data_type, cfg, None, debug=debug)
unet1024.run(data_type, cfg, preds, debug=debug)

util.check_preds_finite(preds)

# Write submission.csv
th = 0.5
submit = write_submission(preds, th, 'submission.csv')

In [ ]:
! tail submission.csv

if data_type == 'validation':
    ! python3 src/score.py submission.csv